In [0]:
import os
import random
import pathlib
import tempfile
import time
import copy
import torch.optim.lr_scheduler as lr_scheduler
from sklearn.metrics import classification_report
import numpy as np
from functools import partial
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset, Subset
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
BASE_DIR= pathlib.Path('/Volumes/pmr_dev/lead_ingots/lead_ingot_images')

image_dir= BASE_DIR / 'Ingot'
label_dir= BASE_DIR / 'Labels/Labels.csv'
file_list = pd.read_csv(label_dir)



In [0]:
display(file_list.head(1))

In [0]:
# Define the function to encode labels
def encode_labels(file_list):
    # Extract image names
    image_names = file_list['Image']
    
    # Combine fault categories into a single label
    # Create a label string by concatenating fault category names where the value is True
    labels = file_list.drop(columns=['Image']).apply(
        lambda row: '_'.join(row.index[row == True]), axis=1
    )
    
    # Encode the labels using LabelEncoder
    le = LabelEncoder()
    encoded_labels = le.fit_transform(labels)
    
    # Create a dictionary mapping image names to encoded labels
    label_dict = dict(zip(image_names, encoded_labels))
    
    return label_dict, le

# Encode the labels
label_dict, label_encoder = encode_labels(file_list)

In [0]:
# Print the mapping of label numbers to fault categories
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Number to Fault Category Mapping:")
for fault_category, label_number in label_mapping.items():
    print(f"{label_number}: {fault_category}")

In [0]:
# Calculate and print the frequencies of the different classes
label_counts = pd.Series(label_dict.values()).value_counts()
print("Frequencies of the different classes:")
print(label_counts)

In [0]:
# Custom Dataset Class
class LeadIngotDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.labels = labels
        self.images = images  
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [0]:
# Load all images at once
def load_images(image_names):
    images = []
    for image_name in image_names:
        image_path = image_dir / image_name
        image = Image.open(image_path).convert('RGB')
        images.append(image)
    return images
# Load all images
image_names = file_list['Image'].tolist()
images = load_images(image_names)

In [0]:
print(len(images))

In [0]:
transform = transforms.Compose([
    transforms.Resize((224, 224)), #alexnet, vgg16 input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) #imagenet noramlization
])

In [0]:
# Create the dataset
labels = [label_dict[img] for img in image_names]
dataset = LeadIngotDataset(images, labels, transform)

In [0]:
batch_size= 32

# Filter images and labels to include only label 7
filtered_indices = [i for i, label in enumerate(labels) if label == 7]
filtered_images = [images[i] for i in filtered_indices]
filtered_labels = [labels[i] for i in filtered_indices]

dataset = LeadIngotDataset(filtered_images,filtered_labels, transform)

train_val_indices, test_indices = train_test_split(
    range(len(dataset)), test_size=0.2, stratify=filtered_labels, random_state=42
)

# Then split train+val into train and validation sets
train_indices, val_indices = train_test_split(
    train_val_indices, test_size=0.125, stratify=[filtered_labels[i] for i in train_val_indices], random_state=42
)

# Create Subset datasets
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

# Print statistics
train_labels = [filtered_labels[i] for i in train_indices]
val_labels = [filtered_labels[i] for i in val_indices]
test_labels = [filtered_labels[i] for i in test_indices]

print(f"Train set size: {len(train_dataset)} frequenties: {torch.bincount(torch.tensor(train_labels))}")
print(f"Validation set size: {len(val_dataset)} frequenties: {torch.bincount(torch.tensor(val_labels))}")
print(f"Test set size: {len(test_dataset)} frequenties: {torch.bincount(torch.tensor(test_labels))}")

In [0]:
import torch
import torch.nn as nn

class UNetAutoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Encoder
        self.encoder1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=2, padding=1),  # -> N, 16, 112, 112
            nn.ReLU()
        )
        self.encoder2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, stride=2, padding=1),  # -> N, 32, 56, 56
            nn.ReLU()
        )
        self.encoder3 = nn.Sequential(
            nn.Conv2d(32, 64, 3, stride=2, padding=1),  # -> N, 64, 28, 28
            nn.ReLU()
        )
        self.encoder4 = nn.Sequential(
            nn.Conv2d(64, 128, 3, stride=2, padding=1),  # -> N, 128, 14, 14
            nn.ReLU()
        )
        self.encoder5 = nn.Sequential(
            nn.Conv2d(128, 256, 3, stride=2, padding=1),  # -> N, 256, 7, 7
            nn.ReLU()
        )
        
        # Decoder with skip connections
        self.decoder5 = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1),  # -> N, 128, 14, 14
            nn.ReLU()
        )
        self.decoder4 = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),  # -> N, 64, 28, 28
            nn.ReLU()
        )
        self.decoder3 = nn.Sequential(
            nn.ConvTranspose2d(128, 32, 3, stride=2, padding=1, output_padding=1),  # -> N, 32, 56, 56
            nn.ReLU()
        )
        self.decoder2 = nn.Sequential(
            nn.ConvTranspose2d(64, 16, 3, stride=2, padding=1, output_padding=1),  # -> N, 16, 112, 112
            nn.ReLU()
        )
        self.decoder1 = nn.Sequential(
            nn.ConvTranspose2d(32, 3, 3, stride=2, padding=1, output_padding=1),  # -> N, 3, 224, 224
            nn.Sigmoid()  # Ensure output is in range [0, 1]
        )
        
    def forward(self, x):
        # Encoding
        enc1 = self.encoder1(x)  # -> N, 16, 112, 112
        enc2 = self.encoder2(enc1)  # -> N, 32, 56, 56
        enc3 = self.encoder3(enc2)  # -> N, 64, 28, 28
        enc4 = self.encoder4(enc3)  # -> N, 128, 14, 14
        enc5 = self.encoder5(enc4)  # -> N, 256, 7, 7
        
        # Decoding with skip connections
        dec5 = self.decoder5(enc5)  # -> N, 128, 14, 14
        dec4 = self.decoder4(dec5)  # Concatenate encoder4 and decoder5 outputs
        dec3 = self.decoder3(torch.cat((dec4, enc3), dim=1))  # -> N, 32, 56, 56
        dec2 = self.decoder2(torch.cat((dec3, enc2), dim=1))  # -> N, 16, 112, 112
        dec1 = self.decoder1(torch.cat((dec2, enc1), dim=1))  # -> N, 3, 224, 224
        
        return dec1


In [0]:
model=UNetAutoencoder()

device = torch.device("cuda:0")
#device = torch.device("cpu")

model = model.to(device)

In [0]:
#x= torch.ones((64,3,224,224))
#model(x).shape

In [0]:
# Define the loss function and optimizer
loss_fn = nn.MSELoss()  # Mean Squared Error loss for autoencoder
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train_autoencoder(model, num_epochs, train_dl, valid_dl, patience):
    train_loss_hist = [0] * num_epochs
    valid_loss_hist = [0] * num_epochs
    patience_counter = 0
    best_model_wts = model.state_dict()

    for epoch in range(num_epochs):
        start_time = time.time()
        print(f"\nEpoch {epoch + 1}/{num_epochs}")

        # Training Phase
        model.train()
        train_loss = 0
        for x_batch, _ in train_dl:
            x_batch = x_batch.to(device)

            # Forward pass
            outputs = model(x_batch)
            loss = loss_fn(outputs, x_batch)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * x_batch.size(0)

        # Calculate average training loss
        train_loss /= len(train_dl.dataset)
        train_loss_hist[epoch] = train_loss

        # Validation Phase
        model.eval()
        valid_loss = 0
        with torch.no_grad():
            for x_batch, _ in valid_dl:
                x_batch = x_batch.to(device)

                # Forward pass
                outputs = model(x_batch)
                loss = loss_fn(outputs, x_batch)

                valid_loss += loss.item() * x_batch.size(0)

        # Calculate average validation loss
        valid_loss /= len(valid_dl.dataset)
        valid_loss_hist[epoch] = valid_loss

        end_time = time.time()
        epoch_duration = end_time - start_time

        # Early Stopping
        if epoch > 0 and valid_loss_hist[epoch] > min(valid_loss_hist[:epoch]):
            patience_counter += 1
        else:
            patience_counter = 0
            best_model_wts = model.state_dict()

        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch + 1}")
            model.load_state_dict(best_model_wts)
            break

        # Epoch Summary
        print(f"Epoch {epoch + 1}/{num_epochs} Summary:")
        print(f"  Training - Loss: {train_loss_hist[epoch]:.4f}")
        print(f"  Validation - Loss: {valid_loss_hist[epoch]:.4f}")
        print(f"  Duration: {epoch_duration:.2f}s")

    return train_loss_hist, valid_loss_hist

# Training parameters
num_epochs = 50
train_loss_hist, valid_loss_hist = train_autoencoder(model, num_epochs, train_loader, val_loader, patience=5)

In [0]:
import matplotlib.pyplot as plt
import numpy as np
x_arr = np.arange(len(train_loss_hist)) + 1

fig = plt.figure(figsize=(12, 4))
ax = fig.add_subplot(1, 2, 1)
ax.plot(x_arr, train_loss_hist, '-o', label='Train loss')
ax.plot(x_arr, valid_loss_hist, '--<', label='Validation loss')
ax.set_xlabel('Epoch', size=15)
ax.set_ylabel('Loss', size=15)


plt.show()

In [0]:
import matplotlib.pyplot as plt

# Function to display images
def denormalize(tensor):
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    return tensor * std + mean
def imshow(img, ax):
    img = denormalize(img)
    npimg = img.numpy()
    ax.imshow(np.transpose(npimg, (1, 2, 0)))

# Get a batch of images from the no_defect_val_loader
dataiter = iter(test_loader)
validation_images, _ = next(dataiter)

# Move the images to the device
validation_images = validation_images.to(device)

# Get the reconstructed images
model.eval()
with torch.no_grad():
    reconstructed = model(validation_images)

# Move the images back to CPU for plotting
validation_images = validation_images.cpu()
reconstructed = reconstructed.cpu()

# Plot the original and reconstructed images
fig, axes = plt.subplots(nrows=2, ncols=6, figsize=(12, 4))

for i in range(6):
    # Original images
    ax = axes[0, i]
    imshow(validation_images[i], ax)
    ax.axis('off')
    if i == 0:
        ax.set_title('Original')

    # Reconstructed images
    ax = axes[1, i]
    imshow(reconstructed[i], ax)
    ax.axis('off')
    if i == 0:
        ax.set_title('Reconstructed')

plt.show()

In [0]:
from torch.utils.data import DataLoader, Subset
import torch

# Filter images and labels to include only label 7
faulty_indices = [i for i, label in enumerate(labels) if label != 7]
print(faulty_indices)
print(len(images))
faulty_images = [images[i] for i in faulty_indices]
faulty_labels = [labels[i] for i in faulty_indices]

faulty_subset = LeadIngotDataset(faulty_images, faulty_labels, transform=transform)

# Create a DataLoader for the faulty subset
faulty_loader = DataLoader(faulty_subset, batch_size=6, shuffle=False)

# Get a batch of images from the no_defect_val_loader
dataiter = iter(faulty_loader)
faulty_images_val, _ = next(dataiter)

# Move the images to the device
faulty_images_val = faulty_images_val.to(device)

# Get the reconstructed images
model.eval()
with torch.no_grad():
    reconstructed = model(faulty_images_val)

# Move the images back to CPU for plotting
faulty_images_val = faulty_images_val.cpu()
reconstructed = reconstructed.cpu()

# Plot the original and reconstructed images
fig, axes = plt.subplots(nrows=2, ncols=6, figsize=(12, 4))

for i in range(6):
    # Original images
    ax = axes[0, i]
    imshow(faulty_images_val[i], ax)
    ax.axis('off')
    if i == 0:
        ax.set_title('Original')

    # Reconstructed images
    ax = axes[1, i]
    imshow(reconstructed[i], ax)
    ax.axis('off')
    if i == 0:
        ax.set_title('Reconstructed')

plt.show()